# Server Module

Copyright 2022 Michael George (AKA Logiqx).

This file is part of [sse-results](https://github.com/Logiqx/sse-results) and is distributed under the terms of the GNU General Public License.

sse-results is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

sse-results is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with sse-results. If not, see <https://www.gnu.org/licenses/>.

In [1]:
import os
import time

import logging
import threading
import queue

## Logging

Initialise the server logging.

In [2]:
def initLogging(level=logging.INFO, filename=None):
    """Initialise logging"""
    
    formatString = "%(asctime)s,%(levelname)s,%(message)s"
    dateTimeFormat = "%Y-%m-%d %H:%M:%S"

    # TODO - implement filename
    logging.basicConfig(format=formatString, datefmt=dateTimeFormat, level=level)
    
    
if __name__ == '__main__':

    # TODO - use app config for level
    initLogging(level=logging.INFO)

## Generic Server Thread and Thread Pool

See https://logiqx.github.io/sse-results/tech/motion.html for further details about the thread model.

In [3]:
class WorkerThread():
    """Generic class for server threads"""

    def __init__(self, name, stopEvent, inQueue, outQueue):
        """Simple initialisation"""

        self.name = name
        self.stopEvent = stopEvent

        self.inQueue = inQueue
        self.outQueue = outQueue

        self.thread = threading.Thread(target=self.loop)


    def start(self):
        """Start the thread"""

        logging.debug("Thread %s starting", self.name)
        self.thread.start()


    def loop(self):
        """Do the actual processing, until the stop event is detected"""

        while not self.stopEvent.is_set():
            try:
                # TODO - config driven timeout
                motion = self.inQueue.get(block=False, timeout=10)

                self.process(motion)
                
                self.outQueue.put(motion)
                
                self.inQueue.task_done()

            except queue.Empty:
                pass
            
            except Exception:
                logging.error("Exception in DetectionThread")


    def process(self, motion):
        """Default behavior suggest the actual process() method is missing"""
        
        logging.warning("Thread %s has an empty process() method", self.name)


    def join(self):
        """Join the thread, essentially waiting for it to stop"""

        self.thread.join()
        logging.debug("Thread %s finished", self.name)

In [4]:
class WorkerThreadPool(WorkerThread):
    """Generic class for a worker thread pool, potentially consisting of a manager thread and multiple worker threads"""

    def __init__(self, name, stopEvent, workerClass, workerCount, inQueue, outQueue):
        """Simple initialisation"""
        
        super().__init__(name, stopEvent, inQueue, outQueue)

        self.workers = []

        for i in range(workerCount):
            worker = workerClass(f'{name} {i}', stopEvent, inQueue, outQueue)
            self.workers.append(worker)


    def start(self):
        """Start the threads"""

        super().start()
        
        for worker in self.workers:
            worker.start()


    def loop(self):
        """There is no processing to be done in an unmanaged pool"""

        pass
    

    def join(self):
        """Join the threads, ensuring they have all stopped cleanly"""
        
        super().join()
        
        for worker in self.workers:
            worker.join()

In [5]:
class SupervisedWorkerThreadPool(WorkerThreadPool):
    """Generic class for a supervised worker thread pool, consisting of a manager thread and multiple worker threads"""
    
    def loop(self):
        """Do the actual processing, until the stop event is detected"""

        while not self.stopEvent.is_set():
            self.process()


    def process(self):
        """Default behavior suggest the actual process() method is missing"""
        
        logging.warning("Thread %s has an empty process() method", self.name)

## Motion Detection Thread and Thread Pool

Detect new motions appearing on the network

In [6]:
class DetectionThread(WorkerThread):
    """Thread pool worker to detect a motion appearing on the network"""

    def process(self, motion):
        """Do the actual processing of a single item"""
        
        logging.info("Thread %s detecting motion %i", self.name, motion)

In [7]:
class DetectionThreadPool(SupervisedWorkerThreadPool):
    """Thread pool manager to detect motions appearing on the network"""

    def process(self):
        """Do the actual processing"""
        
        try:
            # TODO - actual code based on address info being unavailable
            motionCount = 4
            for i in range(motionCount):
                # The manager puts motions onto the in queue for workers to consume
                # logging.info("Queueing %i...", i)
                self.inQueue.put(i)

            # TODO - sensible sleep
            self.stopEvent.wait(5)

        except Exception:
            logging.error("Exception in DetectionThreadPool")


## Motion Inspection Thread

Inspect new motions appearing on the network

In [8]:
class InspectionThread(WorkerThread):
    """Thread pool worker to inspect a motion appearing on the network"""

    def process(self, motion):
        """Do the actual inspection of a single motion"""
        
        logging.info("Thread %s inspecting motion %i", self.name, motion)

## Track Download Thread

Download new tracks for Motion

In [9]:
class DownloadThread(WorkerThread):
    """Thread pool worker to download tracks from a motion"""

    def process(self, motion):
        """Do the actual downloads for a single motion"""
        
        logging.info("Thread %s downloading from motion %i", self.name, motion)

## Motion Disconnect Thread

Detect motions disconnecting from the network

In [10]:
class CleanupThread(WorkerThread):
    """Thread pool worker to detect a motion disappearing from the network"""

    def process(self, motion):
        """Do the actual processing of a single item"""
        
        logging.info("Thread %s checking motion %i", self.name, motion)

In [11]:
class CleanupThreadPool(SupervisedWorkerThreadPool):
    """Thread pool manager to detect motions disappearing on the network"""

    def process(self):
        """Do the actual processing"""
        
        try:
            # TODO - actual code based on address info being unavailable
            motionCount = 4
            for i in range(motionCount):
                # The manager puts motions onto the in queue for workers to consume
                # logging.info("Queueing %i...", i)
                self.inQueue.put(i)

            # TODO - sensible sleep
            self.stopEvent.wait(5)

        except Exception:
            logging.error("Exception in DisconnectionThreadPool")


## Main Function

Startup and shutdown of all threads

In [12]:
def createThreadPools(stopEvent):
    """Create the thread pools, including message queues"""

    threadPools = []

    # Detection
    
    detectionInQueue = queue.Queue()
    detectionOutQueue = queue.Queue()
    
    detectionCount = 2
    detectionThreadPool = DetectionThreadPool(
                            'detect', stopEvent, DetectionThread, detectionCount, detectionInQueue, detectionOutQueue)
    threadPools.append(detectionThreadPool)

    # Inspection
    
    inspectionInQueue = detectionOutQueue
    inspectionOutQueue = queue.Queue()
    
    inspectionCount = 2
    inspectionThreadPool = WorkerThreadPool(
                            'inspect', stopEvent, InspectionThread, inspectionCount, inspectionInQueue, inspectionOutQueue)
    threadPools.append(inspectionThreadPool)

    # Download
    
    downloadInQueue = inspectionOutQueue
    downloadOutQueue = queue.Queue()
    
    downloadCount = 2
    downloadThreadPool = WorkerThreadPool(
                            'download', stopEvent, DownloadThread, inspectionCount, downloadInQueue, downloadOutQueue)
    threadPools.append(downloadThreadPool)

    # Cleanup
    
    cleanupInQueue = queue.Queue()
    cleanupOutQueue = queue.Queue()

    cleanupCount = 2
    cleanupThreadPool = CleanupThreadPool(
                            'cleanup', stopEvent, CleanupThread, cleanupCount, cleanupInQueue, cleanupOutQueue)
    threadPools.append(cleanupThreadPool)


    return threadPools


def main():
    """Main function to start all of the threads"""

    logging.info("Server starting")

    # Global event so that all threads can exit gracefully
    stopEvent = threading.Event()
    
    # Start by creating the thread pools and message queues
    threadPools = createThreadPools(stopEvent)

    # Attempt to start up all of the thread pools
    threadsRunning = False

    try:
        for threadPool in threadPools:
            threadPool.start()

        threadsRunning = True
    
    except Exception:
        logging.error("Thread startup failed")
    
    # Check if thread startup was successful
    if threadsRunning:
        try:
            # Run for a maximum of 30 seconds
            time.sleep(30)
        
        # Intercept keyboard interrupts to ensure that all threads are properly stopped, prior to exit
        except KeyboardInterrupt:
            logging.info("Keyboard interrupt")

    # Raise event so that all threads can exit gracefully
    stopEvent.set()

    # Wait for all active threads to exit gracefully
    for threadPool in threadPools:
        threadPool.join()

    logging.info("Server stopped")


if __name__ == '__main__':
    main()

2022-09-29 19:14:26,INFO,Server starting
2022-09-29 19:14:26,INFO,Thread detect 0 detecting motion 0
2022-09-29 19:14:26,INFO,Thread detect 1 detecting motion 1
2022-09-29 19:14:26,INFO,Thread detect 0 detecting motion 2
2022-09-29 19:14:26,INFO,Thread detect 1 detecting motion 3
2022-09-29 19:14:26,INFO,Thread inspect 0 inspecting motion 0
2022-09-29 19:14:26,INFO,Thread inspect 1 inspecting motion 1
2022-09-29 19:14:26,INFO,Thread cleanup 0 checking motion 0
2022-09-29 19:14:26,INFO,Thread cleanup 1 checking motion 1
2022-09-29 19:14:26,INFO,Thread cleanup 0 checking motion 2
2022-09-29 19:14:26,INFO,Thread cleanup 1 checking motion 3
2022-09-29 19:14:26,INFO,Thread inspect 0 inspecting motion 2
2022-09-29 19:14:26,INFO,Thread inspect 1 inspecting motion 3
2022-09-29 19:14:26,INFO,Thread download 0 downloading from motion 0
2022-09-29 19:14:26,INFO,Thread download 1 downloading from motion 1
2022-09-29 19:14:27,INFO,Thread download 0 downloading from motion 2
2022-09-29 19:14:27,INFO